# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [14]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [15]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [33]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

n = 5
n_arboles = np.random.randint(5, 51, n)
profundidad = np.random.randint(2, 11, n)

lista_f1 = []
for n_arb, prof in zip(n_arboles, profundidad):
    modelo, f1 = Bosque.RegresionBosque(X, y, n_arb, prof)  
    lista_f1.append(f1)

n_arb_vec = n_arboles.reshape([-1, 1])
prof_vec = profundidad.reshape([-1, 1])
X_hiperparam = np.hstack((n_arb_vec, prof_vec))

kernel = 1.0 * RBF(length_scale=1)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp.fit(X_hiperparam, lista_f1)

x_arb = np.linspace(5, 50, 100).reshape([-1, 1])
x_prof = np.linspace(2, 10, 100).reshape([-1, 1])
x_grid_arb, x_grid_prof = np.meshgrid(x_arb, x_prof)
x_h = np.hstack((x_grid_arb.reshape(-1, 1), x_grid_prof.reshape(-1, 1)))

y_pred, y_std = gp.predict(x_h, return_std=True)


max_f1_index = np.argmax(y_pred)
max_f1_value = y_pred[max_f1_index]
optimal_n_arboles = x_h[max_f1_index, 0]
optimal_profundidad = x_h[max_f1_index, 1]

max_f1_value

0.9611168735296758

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

In [32]:
import numpy as np 

X, y = datasets.load_iris(return_X_y=True)
v_arboles=np.arange(5, 51)
v_profundidad=np.arange(2, 11)

T_i=10
T_min=.1
alpha=.9


a_actual=random.choice(v_arboles)
p_actual=random.choice(v_profundidad)
modelo, f1_actual=Bosque.RegresionBosque(X, y, a_actual, p_actual)


def a_new_f1(f1_actual, f1_new, T): 
    if f1_new>f1_actual: 
        return True
    else:
        p=np.exp((f1_new-f1_actual)/T)
        return random.random()<p

T=T_i
mejor_f1=f1_actual
mejor_h=(a_actual, p_actual)
while T>T_min: 
    if random.choice([True, False]): 
        new_a=np.random.choice(v_arboles)
        new_p=p_actual
    else: 
        n_a=a_actual
        n_p=np.random.choice(v_profundidad)
    modelo, n_f1= Bosque.RegresionBosque(X, y, n_a, n_p)
    if a_new_f1(f1_actual, n_f1, T):
        a_actual, p_actual=new_a, new_p
        f1_actual=new_f1
        if n_f1>mejor_f1: 
            mejor_f1=n_f1
            mejor_h=(a_actual, p_actual)
    T*=alpha
mejor_f1

0.9555555555555556

In [30]:
mejor_h[0], mejor_h[1]

(37, 7)